## 引言

通过前面两篇文章[带你实现多头注意力](https://golfxiao.blog.csdn.net/article/details/143697790)和[带你构建TransformerBlock](https://golfxiao.blog.csdn.net/article/details/143741939)，我们基本已经构建完了一个大语言模型的关键模块，本章我们将基于这些模块构建出一个能够运行的GPT Model，并用这个Model类完成从序列文本输入到序列文本输出的整个流程。整个流程如下所示：

![GPT](./img/9-4.png)

- 序列化：用户输入的文本首先通过分词器处理，转化为一系列的token ID；
- 嵌入：离散的token ID序列经过嵌入层（embedding layer）处理后，为每个token生成对应的连续向量表示； 
- 推理：将这些序列向量输入模型，经过一系列连续的矩阵乘法运算，模型将基于已有的上下文信息预测出下一个token的概率分布； 
- 选词：根据概率分布，模型选取下一个最可能的token，从而生成连贯的输出token序列。
- 反序列化：通过分词器的反序列化，将选出的token ID转换回可读的文本形式。

## 模型架构组成

我们将要构建的LLM架构组成如下：

![GPT架构](./img/9-1.jpg)

- 嵌入层（embedding layer）: 用于将tokenID序列转换为连续向量表示； 
- 嵌入丢弃（embedding dropout）: 用于在训练过程中随机丢弃部分嵌入向量，减少模型的过拟合现象； 
- 解码层（decode layers）: 由多个TransformerBlock堆叠而成，负责处理输入并逐步得到序列的上下文向量。
- 最终归一化（final norm）: 对所有输出进行最一化处理，确保输出数值在一个适当的范围内； 
- 输出层（output layer）: 将模型的最终向量转换为token的概率分布； 

下面，我们将一步一步创建出每一层，并用一个输入数据演示每一层运算后的输出效果。

#### 准备工作

首先，我们引入前面已经构建的transformer block组件，其它组件包括多头自注意力也已经包含在此脚本中。

In [2]:
%run transformer.py

定义一套配置项，用于设置模型的基本结构，包括：
- vocab_size: 词表大小, 这里使用分词器训练的词表大小32000; 
- emb_dim: 嵌入层维度，设置为4是为了演示方便；
- n_heads: 多头数量，emb_dim必须是n_heads的整数倍，这里设置为2； 
- context_length: 上下文长度, 为了演示方便，设置为10; 
- n_layers: 解码层数； 
- drop_rate: 训练中dropout层随机丢弃的参数比例； 

In [16]:
config = {
    "vocab_size": 32000, # Vocabulary size
    "context_length": 10, # Context length
    "emb_dim": 4, # Embedding dimension
    "n_heads": 2, # Number of attention heads
    "n_layers": 2, # Number of layers
    "drop_rate": 0.1, # Dropout rate
}

用随机数创建一个batch_size=2, seq_len=4的小批量输入，以这个数据为例来演示模型的前向传播过程。

In [17]:
inputs = torch.randint(low=0, high=config['vocab_size'], size=(2, 4))
b, seq_len = inputs.shape
print(f"batch_size:{b}, seq_len: {seq_len}")
print("inputs:", inputs)

batch_size:2, seq_len: 4
inputs: tensor([[  316, 29259, 15700, 22754],
        [15569, 20794,   313, 14305]])


#### 创建过程

**第一步**：创建嵌入层。

根据配置创建一个词表大小为32000、嵌入维度为4的嵌入层，并对小批量输入序列进行向量嵌入。

In [18]:
token_emb = nn.Embedding(config['vocab_size'], config['emb_dim'])
x = token_emb(inputs)
x

tensor([[[ 1.4290,  0.0106,  1.1043, -0.2775],
         [ 0.0158,  0.8143,  0.9331, -0.9461],
         [-0.5035,  1.7494, -2.2046, -0.2059],
         [-1.0754, -0.0634, -1.4354, -0.2833]],

        [[ 0.5109, -2.1282, -0.3650,  0.0380],
         [-0.8618, -0.9217, -2.3910,  1.7838],
         [ 0.4313, -1.1564,  1.1675,  2.0159],
         [ 0.8844,  2.0213,  2.2478,  0.9087]]], grad_fn=<EmbeddingBackward0>)

**第二步**：创建一个丢弃层，drop_rate=0.1表示每次训练时约有10%的嵌入会被丢弃。
疑问：一个[2,4,4]的数据，为何32个参数只丢弃了1个元素

In [19]:
drop_emb = nn.Dropout(config['drop_rate'])
x = drop_emb(x)
x

tensor([[[ 1.5878,  0.0118,  1.2270, -0.3083],
         [ 0.0176,  0.9047,  1.0368, -1.0512],
         [-0.5595,  1.9437, -2.4496, -0.2288],
         [-1.1949, -0.0704, -1.5949, -0.3148]],

        [[ 0.5676, -2.3646, -0.4056,  0.0422],
         [-0.0000, -1.0241, -2.6567,  1.9820],
         [ 0.4792, -1.2849,  1.2972,  2.2399],
         [ 0.9827,  2.2459,  2.4976,  1.0097]]], grad_fn=<MulBackward0>)

**第三步**：计算位置编码

由于解码层中的注意力得分计算需要使用位置编码，这里先根据`context_length`计算出0-9每个位置的旋转编码，然后截取此次输入长度（seq_len=4)范围内的部分作为输入序列inputs的位置编码。

In [21]:
pos_cis = precompute_pos_cis(config['emb_dim'] // config['n_heads'], config['context_length'])
pos_cis = pos_cis[:seq_len]
pos_cis

tensor([[ 1.0000+0.0000j],
        [ 0.5403+0.8415j],
        [-0.4161+0.9093j],
        [-0.9900+0.1411j]])

**第四步**：创建解码层

根据n_layers参数循环构建出指定数量的解码层，并使用此解码层序列对输入向量作特征计算，最终得到一个能表示输入序列特征的上下文向量。

> 注：每一个解码层是[前文](https://golfxiao.blog.csdn.net/article/details/143741939)讲过的TransformerBlock块实例，里面封装了层归一化、多头注意力、前馈神经网络、残差连接等transformer核心组件。

In [22]:
decode_layers = nn.Sequential(*[
    TransformerBlock(**config) for _ in range(config['n_layers'])
])
for i, block in enumerate(decode_layers):
    x, _ = block(x, pos_cis)
print(x)

tensor([[[ 3.0864, -1.7421,  2.0497, -0.6708],
         [ 1.5507, -0.4535,  1.5019, -2.1026],
         [ 0.3892,  1.4838, -2.0157, -0.7308],
         [-0.5986, -0.9129, -0.9839, -1.0156]],

        [[ 0.3646, -4.1076,  0.5687,  0.2848],
         [-0.2105, -2.0385, -1.7360,  2.0292],
         [-0.5052, -2.8610,  1.8475,  2.1293],
         [ 0.1898,  0.7865,  2.7698,  1.0087]]], grad_fn=<AddBackward0>)



**第五步**：创建最终归一化层，并对解码层计算出的上下文向量进行归一化操作。

In [23]:
final_norm = LayerNorm(config['emb_dim'])
x = final_norm(x)
x

tensor([[[ 1.0632, -1.0708,  0.6050, -0.5974],
         [ 0.8135, -0.3294,  0.7857, -1.2698],
         [ 0.4048,  1.1340, -1.1974, -0.3414],
         [ 1.4614, -0.1840, -0.5557, -0.7217]],

        [[ 0.4810, -1.4979,  0.5713,  0.4457],
         [ 0.1497, -0.8332, -0.6706,  1.3541],
         [-0.2823, -1.2931,  0.7272,  0.8481],
         [-0.9004, -0.3626,  1.4253, -0.1623]]], grad_fn=<AddBackward0>)

> 可以看到，经过归一化后，每个张量的数值范围明显收窄。

**第六步**：创建线性层（Linear Layer），用于将最后的特征映射到词汇表的大小，生成每个token的预测概率。

In [26]:
out_head = nn.Linear(config['emb_dim'], config['vocab_size'])
logits = out_head(x)
logits, logits.shape

(tensor([[[-0.6325,  0.5322, -0.5608,  ...,  0.1865, -1.0097, -0.5029],
          [-0.5466,  0.7494, -0.1760,  ...,  0.1972, -0.9124, -0.3810],
          [-0.0330,  0.1762,  1.0724,  ..., -1.0849,  0.8122, -0.5204],
          [-0.5935,  0.6277,  0.1012,  ..., -0.5491, -0.3485, -0.7212]],
 
         [[-0.3948, -0.0381, -0.7012,  ...,  0.1989, -0.7128, -0.4046],
          [-0.0464, -0.5483, -0.0419,  ..., -0.5780,  0.3811, -0.4891],
          [-0.1028, -0.3863, -0.5254,  ...,  0.2412, -0.3745, -0.2038],
          [ 0.0541, -0.1010, -0.1027,  ...,  0.5089, -0.4024,  0.0881]]],
        grad_fn=<ViewBackward0>),
 torch.Size([2, 4, 32000]))

> 输出的logtis是一个（2，4，32000）形状的张量，表示每个位置的next token预测结果是一个32000维度的向量，即词表中每个token的可能性分数。

## 模型封装

#### 模型配置封装
前面用字典形式创建的模型配置虽然方便，但不够规范，由于兼容transformer库已经成为开源模型的一个事实标准，所以我们也采用结构化的方式来定义模型配置。具体如下 ：
- 使用`transformers.PretrainedConfig`作为模型配置的基类（transformers库的标准）。
- 所有配置参数使用类的成员属性显式定义，这样可以提供默认值，并限制类型。
- 定义一个类属性字段model_type，作为模型独一无二的类型标识，类似通义千问的`qwen`一样，用以和transformers库中其它类型的模型区分。

In [30]:
class GPTConfig(PretrainedConfig):
    # 每个模型都必须有一个独特的model_type，否则会报"Should have a `model_type` key in its config.json"
    model_type = "minigpt"

    def __init__(self, **kwargs):
        self.context_length = kwargs.get('context_length', 1024)
        self.vocab_size = kwargs.get('vocab_size', 32000)
        self.emb_dim = kwargs.get('emb_dim', 768)
        self.drop_rate = kwargs.get('drop_rate', 0.1)
        self.n_layers = kwargs.get('n_layers', 12)
        self.n_heads = kwargs.get('n_heads', 12)
        self.qkv_bias = kwargs.get('qkv_bias', False)
        super().__init__(**kwargs)

cfg = GPTConfig()
cfg

GPTConfig {
  "context_length": 1024,
  "drop_rate": 0.1,
  "emb_dim": 768,
  "model_type": "minigpt",
  "n_heads": 12,
  "n_layers": 12,
  "qkv_bias": false,
  "transformers_version": "4.44.2",
  "vocab_size": 32000
}

> 上面的各个配置项的默认值，就是我们将要构建的模型目标结构，这里采用768维的向量嵌入，12个解码层，12个注意力头，1024的上下文长度。

#### 模型结构封装

我们最终创建一个名为MiniGPT的模型类，为了与HuggingFace的transformers库兼容，该类需要满足两点：
1. 继承自PreTrainedModel
2. 采用PreTrainedConfig类型的配置对象作为构造函数参数。

> 标准化的基类还提供了以下好处：
> - 可以直接使用from_pretrained和save_pretrained方法来加载模型权重和保存模型权重，不用关心权重存储细节； 
> - 可以集中管理与模型结构相关的超参数，使得模型的构建代码非常简洁一致，与便于配置的扩展。

具体操作就是将上面嵌入层、dropout层、解码层、最终归一化层、输出层的创建代码封装到构造方法中。

In [ ]:
class MiniGPT(PreTrainedModel):
    config_class = GPTConfig

    def __init__(self, config: GPTConfig):
        super().__init__(config)
        self.context_length = config.context_length
        self.num_heads = config.n_heads
        self.n_layers = config.n_layers
        self.token_emb = nn.Embedding(config.vocab_size, config.emb_dim)
        self.drop_emb = nn.Dropout(config.drop_rate)
        
        pos_cis = precompute_pos_cis(config.emb_dim // config.n_heads, config.context_length)
        self.register_buffer("pos_cis", pos_cis, persistent=False)
        self.decode_layers = nn.Sequential(*[
            TransformerBlock(**(config.to_dict())) for _ in range(config.n_layers)
        ])
        
        self.final_norm = LayerNorm(config.emb_dim)
        self.out_head = nn.Linear(config.emb_dim, config.vocab_size)

#### 模型推理封装
将前面各个层的运算代码封装到forward方法中。

In [31]:
def forward(self, inputs:torch.Tensor, **kwargs):
    b, seq_len = inputs.shape
    pos_cis = self.pos_cis[:seq_len]
    x = self.token_emb(inputs)
    x = self.drop_emb(x)

    for i, block in enumerate(self.decode_layers):
        x, _ = block(x, pos_cis)

    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits

setattr(MiniGPT, "forward", forward)


> Tips1：定义模型对外的forward时，最好预留一个kwargs参数，用于接受一些训练器额外传递的参数，例如注意力掩码、kv_cache开关等，如果没有kwargs，python方法接收未声明的参数时会报错。

> Tips2：很多文章都提到dropout只会用在训练模式下，但上面的drop_emb方法我们并不需要显式加模式判断，原因在于Dropout组件内部实现了train()和eval()模式的判断，只有在train模式下才会进行随机丢弃。有相同行为的组件还有BatchNorm，只有train模式下才会使用批次统计信息，推理模式下会使用整个训练数据的统计信息。

#### 模型测试
下面我们初始化这个模型，以随机初始化的批次序列来查看模型的输出。

In [32]:
torch.manual_seed(123)
batch = torch.randint(low=0, high=cfg.vocab_size, size=(2, 4))
model = MiniGPT(cfg)
logits = model(batch)
print("inputs: ", batch)
print("output shape:", logits.shape)
print("outputs:", logits)

inputs:  tensor([[16382,  7789, 31102, 12610],
        [15580, 31842,  6886, 23057]])
output shape: torch.Size([2, 4, 32000])
outputs: tensor([[[ 0.6840,  0.1197,  0.1772,  ...,  0.1654,  0.7179, -0.1576],
         [ 0.3972,  0.2587,  0.8051,  ...,  0.4466,  0.2240,  0.0966],
         [-0.3390, -0.3504, -0.2874,  ..., -0.4336,  1.0594, -0.5151],
         [ 0.4777,  0.1758, -0.6122,  ..., -0.4519,  0.0900,  0.2798]],

        [[ 0.3365, -0.2321,  0.2035,  ..., -1.0596, -0.4203,  0.5169],
         [-0.0364, -0.4018,  0.4191,  ..., -0.4225,  0.3543, -0.3469],
         [ 0.5884, -0.4254, -0.1675,  ..., -0.8440,  0.1559, -0.3888],
         [ 0.7392,  0.2481,  1.1377,  ..., -0.1336, -0.1802,  0.0230]]],
       grad_fn=<ViewBackward0>)


通过`model.parameters()`可以计算模型的参数量。

In [34]:
total_params = sum(param.numel() for param in model.parameters())
total_params

134212352

此模型有1.34亿个参数。以32位浮点数精度来计算模型参数的内存需求：

In [35]:
total_size_bytes = total_params * 4
print(f"total_size of the model: {total_size_bytes/(1024*1024):.2f}MB")

total_size of the model: 511.98MB


## 生成文本序列

我们对模型的需求是输出一个完整的文本序列，而上面构建的MiniGPT仅仅是输出下一个token的logits。为了生成一个序列我们需要对GPTModel进行多次迭代调用，每次迭代得到一个token，再把这个token添加到序列中继续迭代，整个过程类似下图所示。
![LLM模型](./img/9-2.jpg)

为此，需要编写一个generate函数来完成一个序列的预测。

In [36]:
@torch.inference_mode
def generate(self, input_ids, max_length=512, pad_token_id=-1):
    # 创建batch长度的全零值，用作停止推理的判断。
    eos_reached = torch.zeros(len(input_ids), dtype=torch.bool, device=input_ids.device)
    for _ in range(max_length):
        # 如果生成序列过程中超出上下文长度，则由后往前截取context_length个token。
        context_ids = input_ids[:, -self.context_length:]  
        with torch.no_grad():
            output = self(context_ids)  # shape: batch, n_tokens, vocab_size

        # 只取每个序列最后一个token的输出向量作为logits, shape变为: batch, vocab_size
        logits = output[:, -1, :]        
        # 使用softmax函数将logits转换为下一个token的概率分布，shape仍是: batch, vocab_size
        probs = torch.softmax(logits, dim=-1)   
        # 取概率最大的作为next_token_ids，形状变为：batch, 1
        next_token_ids = torch.argmax(probs, dim=-1, keepdim=True)
        # 将next_token_id连接到下一个token的结尾， 形状变为：batch, n_tokens+1
        input_ids = torch.cat((input_ids, next_token_ids), dim=1)
        
        # 更新 eos_reached，需要所有batch都推理出eos_token，才会终止推理
        eos_reached |= (next_token_ids.squeeze(-1) == pad_token_id)
        if eos_reached.all(): break

    return input_ids

setattr(MiniGPT, "generate", generate)

上面函数中for循环内部的代码逻辑，是在完成一个token的预测，代码逻辑可以按照下图所示的流程来辅助理解：
![next token预测](./img/9-3.jpg)

在测试这个generate函数之前，需要先创建一个Tokenizer类，用于文本到序列的转换，这里使用之前训练好的分词器`tokenizer_v3`。

In [37]:
from transformers import AutoTokenizer

tokenizer_path = "/data2/minigpt/models/tokenizer_v3"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

input_text = "Hello, I am"
input_ids = tokenizer.encode(input_text)
input_ids

[10467, 14, 384, 1207]

接下来，就用这个输入序列来测试generate函数。

In [40]:
batch = torch.tensor(input_ids).unsqueeze(0)
eos_token = tokenizer.eos_token_id
generated_seqs = model.generate(batch, 5, tokenizer.eos_token_id)
print("inputs: ", batch)
print("output:", generated_seqs)

inputs:  tensor([[10467,    14,   384,  1207]])
output: tensor([[10467,    14,   384,  1207,  6680, 22574,  5832, 25201, 26875]])


> unsqueeze方法用于扩展维度，上面的代码是按照指定的维度`0`将张量由形状为[6]->[1, 6]，扩展维度的原因是模型只接受批量输入，这里即使只有一个序列，也需要将张量扩展成批次输入的形状。

下面使用decode方法将输出的数字序列转换为文本。

In [41]:
tokenizer.decode(generated_seqs.squeeze(0))

'Hello, I am危险体会参数的液iform'

> squeeze方法用于压缩维度，上面的示例中是将第0维去掉，张量形状由[1, 11]->[11]。需要注意的是，squeeze只能对size=1的维度进行操作，如果对size不等于1的维度操作将不会有任何改变。

由于模型还没有经过预训练，所以目前模型输出的token都是随机的，没有任何含义。

**小结**：本文从GPT模型的结构说明开始，一步一步创建了模型推理过程中用到的每个组件，并演示了每个组件对输入数据运算的效果。随后基于这些组件封装出了我们自己的模型类MiniGPT，并基于这个模型类进行了自回归生成文本序列的演示，目前生成的序列是没有含义的，需要对这个模型进行训练后才能像GPT一样生成有含义的文本。